In [1]:
import uuid
import pyspark.sql.functions as F
from pyspark.sql.functions import col
from pyspark.sql.types import StringType,DateType,LongType,IntegerType,TimestampType

#JDBC connect details for SQL Server database
jdbcHostname = "jdbcHostname"
jdbcDatabase = "jdbcDatabase"
jdbcUsername = "jdbcUsername"
jdbcPassword = "jdbcPassword"
jdbcPort = "1433"

connectionProperties = {
  "user" : jdbcUsername,
  "password" : jdbcPassword,
  "driver" : "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}

jdbcUrl = "jdbc:sqlserver://{0}:{1};database={2};user={3};password={4}".format(jdbcHostname, jdbcPort, jdbcDatabase, jdbcUsername, jdbcPassword)
writeConfig = {
    "Endpoint": "Endpoint",
    "Masterkey": "Masterkey",
    "Database": "Movies",
    "Collection": "Orders",
    "Upsert": "true"
}

In [2]:
import json
import ast
import pyspark.sql.functions as F
from azure.cosmos import exceptions, CosmosClient, PartitionKey
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark.sql import *
from pyspark.sql.functions import exp
from pyspark.sql.functions import col
from pyspark.sql.functions import lit
from pyspark.sql.functions import array
from pyspark.sql.types import *

#get all orders
orders = sqlContext.read.jdbc(url=jdbcUrl, table="orders", properties=connectionProperties)

#get all order details
orderdetails = sqlContext.read.jdbc(url=jdbcUrl, table="orderdetails", properties=connectionProperties)

#filter on OrderId to loop
cols = orders.select('OrderId').collect()

counter = 1
for col in cols:
  #filter the order on current loop
  order = orders.filter(orders['OrderId'] == col[0])
  
  #set id to be iterator number
  order = order.withColumn("id", lit(str(counter)))
  
  #add details field to order dataframe
  order = order.withColumn("details", lit(0))
  
  #filter order details dataframe to get details we want to merge into the order document
  orderdetailsgroup = orderdetails.filter(orderdetails['OrderId'] == col[0])
  
  #convert dataframe to pandas because of trouble putting array type into spark dataframe field
  orderpandas = order.toPandas()
  
  #convert details dataframe to json, but only if details were returned
  if (orderdetailsgroup.count() !=0):
    jsonstring = orderdetailsgroup.toJSON().collect()
    
    #set details field to be the details json array
    orderpandas['details'][0] = jsonstring    
  
  #convert the order dataframe to json do some string manipulation to get valid json
  orderjson = orderpandas.to_json(orient='records')
  results = orderjson
  results = results.replace("\\", "")
  results = results.replace("[\"", "[")
  results = results.replace("\"]", "]")
  results = results.replace("}\",\"{", "},{")
  results = results.replace("\"", "'")
  results = results[1:-1] 
  
  #read json into spark dataframe
  df = spark.read.json(sc.parallelize([results]))
  
  #write the dataframe (single order record with merged many-to-one order details) to cosmos db using spark connector
  df.write.format("com.microsoft.azure.cosmosdb.spark").mode("append").options(**writeConfig).save()
  
  #increment the counter
  counter += 1

/local_disk0/tmp/1575531358606-0/PythonShell.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
 from IPython.utils.strdispatch import StrDispatch